In [1]:
from utils import DocumentDB
db = DocumentDB()
print("loaded")

loaded


In [2]:
from src import SemanticIndex, BM25Index

semantic_idx = SemanticIndex()
bm25_idx = BM25Index()

print("loaded")

/home/dhairya.dalal/.conda/envs/nlp-env/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0
loaded


In [3]:
import pytrec_eval
from utils import read_qrel_from_file, evaluate_run, extract_topics_from_file
import re
import numpy as np
from nltk.tokenize import sent_tokenize

topics = extract_topics_from_file("qrels/2020/topics_test.txt")
qrel = read_qrel_from_file("qrels/2020/cair2020_qrel.txt")

metrics: set = {'map', 'ndcg', 'P_5'}
evaluator = pytrec_eval.RelevanceEvaluator(qrel, metrics)

causitives = ["because", "because of", "due to", "lead to", "causes", "result of", "leading cause"]
causitive_re = re.compile(r"|".join(causitives))

ignore = ["irrelevant", "not relevant", "does not satisfy relevance"]
ignore_re = re.compile(r"|".join(ignore))

def print_cum_stats(run):
    run_results = evaluator.evaluate(run)

    map_scores = [v["map"] for k,v in run_results.items()]
    p_scores  = [v["P_5"] for k,v in run_results.items()]
    ndcg_scores = [v['ndcg'] for k,v in run_results.items()]

    print("Aggregate results")
    print("Average MAP: ", np.mean(map_scores))
    print("Average P_5: ", np.mean(p_scores))
    print("Average NDCG: ", np.mean(ndcg_scores))


In [4]:
import pke
from nltk.corpus import stopwords

def extract_key_words(text):
    extractor = pke.unsupervised.TopicRank()
    extractor.load_document(input=text, language='en')
    extractor.candidate_selection()
    extractor.candidate_weighting(threshold=.65)
    kw = extractor.get_n_best(n=15)
    return " ".join([tup[0] for tup in kw])

In [5]:
from collections import Counter
from tqdm.notebook import tqdm
title_run = {}
narrative_run = {}
combined_run = {}

NUM_RESULTS = 1000
for topic in tqdm(topics, total = len(topics)):
    # Title results 
    title = topic["title"].strip()
    sti, sts = semantic_idx.query(title, num_results=NUM_RESULTS)
    stid = db.batch_idx_lookup(sti)
    stid_scores = dict(zip(stid, sts))

    bti, bts = bm25_idx.query(title, num_results=NUM_RESULTS)
    btid = db.batch_idx_lookup(bti)
    btid_scores = dict(zip(btid, bts))

    tctr = Counter(stid+btid)
    t_scores = {}
    for i in tctr:
        s_score = stid_scores.get(i, 0)
        b_score = btid_scores.get(i, 0)
        t_scores[i] = s_score + b_score
 

    # Narrative results 
    narrative = topic["narrative"].strip()
    rel_narrative = [sent for sent in sent_tokenize(narrative) if not ignore_re.search(sent) ]
    rel_narrative = " ".join(rel_narrative)
    
    narr_keywords = extract_key_words(rel_narrative)
    
    # sni, sns = semantic_idx.query(narr_keywords, num_results=30)
    # snid = db.batch_idx_lookup(sni)
    # snid_scores = dict(zip(snid, sts))

    bni, bns = bm25_idx.query(narr_keywords, num_results=NUM_RESULTS)
    bnid = db.batch_idx_lookup(bni)
    bnid_scores = dict(zip(bnid, bts))

    # nctr = Counter(snid+bnid)
    # n_scores = {}
    # for i in nctr:
    #     s_score = snid_scores.get(i, 0)
    #     b_score = bnid_scores.get(i, 0)
    #     n_scores[i] = s_score + b_score

    # combined_scores = {}
    # all_keys = list(n_scores.keys()) + list(t_scores.keys())
    # for key in all_keys:
    #     t_s = t_scores.get(key, 0)
    #     n_s = n_scores.get(key,0)
    #     combined_scores[key] = t_s + n_s

    combined_scores = {}
    all_keys = list(t_scores.keys()) + list(bnid_scores.keys())
    for key in all_keys:
        t_s = t_scores.get(key, 0)
        n_s = bnid_scores.get(key,0)
        combined_scores[key] = np.mean([t_s,n_s])
    combined_run[topic["number"]] = combined_scores

print_cum_stats(combined_run)


Aggregate results
Average MAP:  0.5774989891215204
Average P_5:  0.78
Average NDCG:  0.8580313074222803


In [8]:
from sentence_transformers import SentenceTransformer, CrossEncoder, util
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')


In [45]:
from collections import Counter
from tqdm.notebook import tqdm
title_run = {}
narrative_run = {}
combined_run = {}

NUM_RESULTS = 100
for topic in tqdm(topics, total = len(topics)):
    # Title results 
    title = topic["title"].strip()
    sti, sts = semantic_idx.query(title, num_results=NUM_RESULTS)
    stid = db.batch_idx_lookup(sti)
    stid_scores = dict(zip(stid, sts))

    bti, bts = bm25_idx.query(title, num_results=NUM_RESULTS)
    btid = db.batch_idx_lookup(bti)
    btid_scores = dict(zip(btid, bts))

    tctr = Counter(stid+btid)
    t_scores = {}
    for i in tctr:
        s_score = stid_scores.get(i, 0)
        b_score = btid_scores.get(i, 0)
        t_scores[i] = s_score + b_score
 

    # Narrative results 
    narrative = topic["narrative"].strip()
    rel_narrative = [sent for sent in sent_tokenize(narrative) if not ignore_re.search(sent) ]
    rel_narrative = " ".join(rel_narrative)
    
    narr_keywords = extract_key_words(rel_narrative)
    
    # sni, sns = semantic_idx.query(narr_keywords, num_results=30)
    # snid = db.batch_idx_lookup(sni)
    # snid_scores = dict(zip(snid, sts))

    bni, bns = bm25_idx.query(narr_keywords, num_results=NUM_RESULTS)
    bnid = db.batch_idx_lookup(bni)
    bnid_scores = dict(zip(bnid, bts))

    # nctr = Counter(snid+bnid)
    # n_scores = {}
    # for i in nctr:
    #     s_score = snid_scores.get(i, 0)
    #     b_score = bnid_scores.get(i, 0)
    #     n_scores[i] = s_score + b_score

    # combined_scores = {}
    # all_keys = list(n_scores.keys()) + list(t_scores.keys())
    # for key in all_keys:
    #     t_s = t_scores.get(key, 0)
    #     n_s = n_scores.get(key,0)
    #     combined_scores[key] = t_s + n_s

    #combined_scores = {}
    all_keys = list(t_scores.keys()) + list(bnid_scores.keys())
    # for key in all_keys:
    #     t_s = t_scores.get(key, 0)
    #     n_s = bnid_scores.get(key,0)
    #     combined_scores[key] = np.mean([t_s,n_s])
    # combined_run[topic["number"]] = combined_scores

    all_keys = list(set(all_keys))
    docs = db.batch_docno_lookup(all_keys)
    inputs = [title] * len(docs)
    pairs  = list(zip(inputs, docs))
    preds = cross_encoder.predict(pairs,  show_progress_bar=True)
    #preds = [max(v, 0) for v in preds.tolist()]
    preds = preds.tolist()
    
    combined_scores = dict(zip(all_keys, preds))
    combined_run[topic["number"]] = combined_scores

print_cum_stats(combined_run)



Aggregate results
Average MAP:  0.48283078481786
Average P_5:  0.6
Average NDCG:  0.7545790605932867


In [ ]:
Aggregate results
Average MAP:  0.4106417897733161
Average P_5:  0.52
Average NDCG:  0.707158510865819

In [ ]:
Aggregate results
Average MAP:  0.48283078481786
Average P_5:  0.6
Average NDCG:  0.7545790605932867

In [46]:
from transformers import AutoTokenizer

In [47]:
tokenizer = AutoTokenizer.from_pretrained("cross-encoder/ms-marco-MiniLM-L-6-v2")

In [48]:
tokenizer.encode("testing")

[101, 5604, 102]

In [49]:
tokenizer.decode([101, 5604, 102])

'[CLS] testing [SEP]'